##### MDOEL EVALUATION AND MLFLOW INTEGRATION TO TRACK EXPERIMENTS

In [1]:
import os

print(f"Original Working Directory:{os.getcwd()}")

os.chdir("../")

#print(f"Current Working Directory after Changed: {os.getcwd()}")

Original Working Directory:c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification\research
Current Working Directory after Changed: c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification


###### GET DETAILS BELOW FROM THE DAGSHUB.COM AFTER YOU HAVE CREATED A NEW PROJECT AND LINK WITH YOUR GITHUB

`os.environ["MLFLOW_TRACKING_URL"]="https://dagshub.com/USER_NAME/PROJECT_TITLE.mlflow"`

`os.environ["MLFLOW_TRACKING_USERNAME"]="USER_NAME"`

`os.environ["MLFLOW_TRACKING_PASSWORD"]="40_HASH_TOKENS"`

In [34]:
# LOAD THE MODEL
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/trained_model.h5")
model

<Functional name=functional, built=True>

In [ ]:
# UPDATE ENTITY - AFTER THE CONFIG AND PARAMS ARE UPDATED

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    trained_model_path: Path
    training_data: Path
    scores_file_path: Path
    all_params: dict
    #mlflow_url: str
    params_augmentation: bool
    params_image_size: list
    params_batch_size: int
    params_epochs: int



In [ ]:
# UPDATE THE CONFIGURATION INSIDE THE SRC/CONFIG - THIS IS ESSENTIAL AS THE CURRENT WORKFLOW WILL USE IT TO RUN

from src.Kidney_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

from Kidney_classifier.utils.common import read_yaml_file, create_directories, save_json_file


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)

        create_directories([self.config.artifacts_root])

    
    # def get_evaluation_config(self) -> ModelEvaluationConfig:
        
    #     eval_config = ModelEvaluationConfig(
    #         trained_model_path="artifacts/training/trained_model.h5",
    #         training_data="artifacts/data_ingestion/kidney_disease_scan_image",
    #         mlflow_url="https://dagshub.com/USER_NAME/PROJECT_TITLE.mlflow", # change this to the actual repo name
    #         all_params = self.params.prepare_base_model,
    #         params_augmentation=self.params.prepare_base_model.AUGMENTATION,
    #         params_image_size=self.params.prepare_base_model.IMAGE_SIZE,
    #         params_batch_size=self.params.prepare_base_model.BATCH_SIZE,
    #         params_epochs=self.params.prepare_base_model.EPOCHS
    #     )

    #     return eval_config
    
    
    def get_evaluation_config(self) -> ModelEvaluationConfig:
        all_eval_config = self.config.model_evaluation
        all_eval_params = self.params.prepare_base_model

        eval_config = ModelEvaluationConfig(
            trained_model_path=Path(all_eval_config.trained_model_path),
            training_data=Path(all_eval_config.training_data),
            scores_file_path=Path(all_eval_config.scores_file_path),
            all_params = self.params.prepare_base_model,
            #mlflow_url=all_eval_config.mlflow_url,
            params_augmentation=all_eval_params.AUGMENTATION,
            params_image_size=all_eval_params.IMAGE_SIZE,
            params_batch_size=all_eval_params.BATCH_SIZE,
            params_epochs=all_eval_params.EPOCHS
        )

        return eval_config

In [25]:
# UPDATE THE COMPONENTS - THIS IS WHERE YOU CREATE THE FILE FOR EVALUATING THE MODEL

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from Kidney_classifier import logger

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def _testing_validation_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        validation_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.validation_generator = validation_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_trained_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def model_evaluation(self):
        self.model = self.load_trained_model(self.config.trained_model_path)
        self._testing_validation_generator()
        self.score = model.evaluate(self.validation_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        #save_json_file(path=Path("scores.json"), data=scores)
        self.config.scores_file_path.parent.mkdir(parents=True, exist_ok=True)
        save_json_file(path=self.config.scores_file_path, data=scores)


    
    # MLFLOW EXPERIMENT TRACKING CODE SNIPET
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model_as_base_transfer_learning")
            else:
                mlflow.keras.log_model(self.model, "model")


In [27]:
# UPDATE THE PIPELINE

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.model_evaluation()
    #model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 40 images belonging to 2 classes.


c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification\KIDNEY_CLASS_VENV\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9750 - loss: 0.2282


In [31]:
""" # CODE REFACTORED FOR MODEL EVALUATION AND MLFLOW

# MODEL EVALUATION

class ModelEvaluation2:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.model = None
        self.score = None
        self.validation_generator = None

    def _testing_validation_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        validation_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.validation_generator = validation_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_trained_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def model_evaluate(self):
        
        self.model = self.load_trained_model(self.config.trained_model_path)
        self._testing_validation_generator()
        self.score = self.model.evaluate(self.validation_generator)
        self.save_score()
        return self.model, self.score

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        #save_json_file(path=Path("scores.json"), data=scores)
        self.config.scores_file_path.parent.mkdir(parents=True, exist_ok=True)
        save_json_file(path=self.config.scores_file_path, data=scores)
"""

' # CODE REFACTORED FOR MODEL EVALUATION AND MLFLOW\n\n# MODEL EVALUATION\n\nclass ModelEvaluation2:\n    def __init__(self, config: ModelEvaluationConfig):\n        self.config = config\n        self.model = None\n        self.score = None\n        self.validation_generator = None\n\n    def _testing_validation_generator(self):\n        datagenerator_kwargs = dict(\n            rescale=1./255,\n            validation_split=0.10\n        )\n\n        dataflow_kwargs = dict(\n            target_size=self.config.params_image_size[:-1],\n            batch_size=self.config.params_batch_size,\n            interpolation="bilinear"\n        )\n\n        validation_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)\n\n        self.validation_generator = validation_datagenerator.flow_from_directory(\n            directory=self.config.training_data,\n            subset="validation",\n            shuffle=False,\n            **dataflow_kwargs\n        )\n\n    

In [32]:
""" # UPDATE MLFLOW 

import mlflow
import mlflow.keras
from urllib.parse import urlparse

class MLflowLogger:
    def __init__(self, config):
        self.config = config

    def log_metrics_and_model(self, model, score):
        
        "Logs parameters, metrics, and model into MLflow."
        
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Logging hyperparameters
            mlflow.log_params(self.config.all_params)

            # Logging evaluation metrics
            mlflow.log_metrics({"loss": score[0], "accuracy": score[1]})

            # Logging and optionally register the model
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    model,
                    "model",
                    registered_model_name="VGG16Model_as_base_transfer_learning"
                )
            else:
                mlflow.keras.log_model(model, "model")
"""


' # UPDATE MLFLOW \n\nimport mlflow\nimport mlflow.keras\nfrom urllib.parse import urlparse\n\nclass MLflowLogger:\n    def __init__(self, config):\n        self.config = config\n\n    def log_metrics_and_model(self, model, score):\n\n        "Logs parameters, metrics, and model into MLflow."\n\n        mlflow.set_registry_uri(self.config.mlflow_url)\n        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme\n\n        with mlflow.start_run():\n            # Logging hyperparameters\n            mlflow.log_params(self.config.all_params)\n\n            # Logging evaluation metrics\n            mlflow.log_metrics({"loss": score[0], "accuracy": score[1]})\n\n            # Logging and optionally register the model\n            if tracking_url_type_store != "file":\n                mlflow.keras.log_model(\n                    model,\n                    "model",\n                    registered_model_name="VGG16Model_as_base_transfer_learning"\n                )\n          

In [11]:
""" # Assuming ModelEvaluationConfig is defined somewhere
config = ModelEvaluationConfig()

# Step 1 — Evaluate model
evaluator = ModelEvaluation2(config)
model, score = evaluator.evaluate()

# Step 2 — Log results to MLflow
mlflow_logger = MLflowLogger(config)
mlflow_logger.log_metrics_and_model(model, score)
"""


' # Assuming ModelEvaluationConfig is defined somewhere\nconfig = ModelEvaluationConfig()\n\n# Step 1 — Evaluate model\nevaluator = ModelEvaluation2(config)\nmodel, score = evaluator.evaluate()\n\n# Step 2 — Log results to MLflow\nmlflow_logger = MLflowLogger(config)\nmlflow_logger.log_metrics_and_model(model, score)\n'

In [33]:
""" # UPDATE PIPELINE

try:
    # Step 1: Load all configurations
    config = ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()

    # Step 2: Evaluate model
    model_evaluator = ModelEvaluation2(config=model_evaluation_config)
    model, score = model_evaluator.evaluate()

    # Step 3: Log results to MLflow
    mlflow_logger = MLflowLogger(config=model_evaluation_config)
    mlflow_logger.log_metrics_and_model(model, score)

except Exception as e:
    raise e

"""


' # UPDATE PIPELINE\n\ntry:\n    # Step 1: Load all configurations\n    config = ConfigurationManager()\n    model_evaluation_config = config.get_evaluation_config()\n\n    # Step 2: Evaluate model\n    model_evaluator = ModelEvaluation2(config=model_evaluation_config)\n    model, score = model_evaluator.evaluate()\n\n    # Step 3: Log results to MLflow\n    mlflow_logger = MLflowLogger(config=model_evaluation_config)\n    mlflow_logger.log_metrics_and_model(model, score)\n\nexcept Exception as e:\n    raise e\n\n'